In [1]:
# Program prepares a list of option stocks and indexes for IBKR
# Can be run once every month

# STATUS: Complete
# Runtime: 55 secs

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=2) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=2) # kavi IBG live

API connection failed: ConnectionRefusedError(10061, "Connect call failed ('127.0.0.1', 7496)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 10061] Connect call failed ('127.0.0.1', 7496)

In [ ]:
%%time
import pandas as pd
import sys

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
market = 'NSE'

#*******      Symbol extraction and correction    ***
#____________________________________________________

df = pd.DataFrame([]) # Initialize

#IBKR's symbols page
url = "https://www.interactivebrokers.com.hk/en/index.php?"
url = url+"f=2222&exch=nse&showcategories=OPTGRP&page="

for i in range(4):
    df = df.append(df.append(pd.read_html(url+str(i), header=0)[2]))

# Exit program if dataframe is empty
if df.empty:
    print('Interactive Brokers dataframe is empty!!!')
    sys.exit(0)
    
df = df.drop_duplicates().reset_index(drop=True)

# Qualify Stock contracts
stocks = [Stock(symbol=s, exchange=market) for s in df['IB Symbol']]
nse_qual_stocks = ib.qualifyContracts(*stocks)

# Make the dataframe
df_qual_stocks = pd.DataFrame([(s.symbol, s.conId, s.exchange, 
                             s.currency, s.localSymbol) 
                             for s in nse_qual_stocks], 
                             columns=['ibSymbol', 'conId', 'exchange', 
                              'currency', 'nseSymbol'])

# Pickle the equity symbols
df_qual_stocks.to_pickle(datapath+r'df_nse_eq_symbols.pkl')

df.head()

# Eliminate successful stocks to make index list
idx_list1 = list(df[~df['IB Symbol'].isin(df_qual_stocks.ibSymbol)]['IB Symbol'].values)

# Add other indexes to the list
idx_list = idx_list1 + ['NIFTY', 'FTSE100', 'NIFTYIT', 'NIFTYMID50', 'INDIAVIX', 
              'NIFTYCPSE', 'NIFTYPSE', 'NIFTYINFRA']

# Qualify Index contracts
indices = [Index(symbol=s, exchange=market) for s in idx_list]
nse_qual_indices = ib.qualifyContracts(*indices)

# Make the dataframe
df_qual_indices = pd.DataFrame([(i.symbol, i.conId, i.exchange, i.currency, i.localSymbol) 
              for i in nse_qual_indices],
              columns=['ibSymbol', 'conId', 'exchange', 'currency', 'nseSymbol'])

# Replace NIFTY50 with NIFTY for NSE web extraction
df_qual_indices.loc[df_qual_indices.nseSymbol == 'NIFTY50', 'nseSymbol'] = 'NIFTY'

# Pickle the index symbols
df_qual_indices.to_pickle(datapath+r'df_nse_idx_symbols.pkl')

ib.disconnect()